In [15]:
show_animation = True
hide_plots = False


# importing libraries to use (libraries contain code shortcuts)
import random
import simpy # DES
import numpy as np # a maths and plotting module
import pandas as pd # more data analysis
import matplotlib.pyplot as plt # 
import seaborn as sns
import math
import time
from tkinter import *


################ SET UP ANIMATION CANVAS #################
class penny:
    def __init__(self, canvas, x1, y1, x2, y2, tag):
        self.x1 = x1
        self.y1 = y1
        self.x2 = x2
        self.y2 = y2
        self.canvas = canvas
        self.penny = canvas.create_oval(self.x1, self.y1, self.x2, self.y2, fill="red", tags = tag)
        self.penny_number = canvas.create_text(((self.x2 - self.x1)/2 + self.x1), ((self.y2 - self.y1)/2 + self.y1), text = tag) #******tag
        self.canvas.update()

    def move_penny(self, deltax, deltay):
        self.canvas.move(self.penny, deltax, deltay)
        self.canvas.move(self.penny_number, deltax, deltay)
        self.canvas.update()
        
    def remove_penny(self):
        self.canvas.delete(self.penny)
        self.canvas.delete(self.penny_number)
        self.canvas.update()

class Clock:
    def __init__(self, canvas, x1, y1, x2, y2, tag):
        self.x1 = x1
        self.y1 = y1
        self.x2 = x2
        self.y2 = y2
        self.canvas = canvas
        self.penny = canvas.create_rectangle(self.x1, self.y1, self.x2, self.y2, fill="#fff")
        self.time = canvas.create_text(((self.x2 - self.x1)/2 + self.x1), ((self.y2 - self.y1)/2 + self.y1), text = "Time = "+str(tag)+"s")
        self.canvas.update()

    def tick(self, tag):
        self.canvas.delete(self.time)
        self.time = canvas.create_text(((self.x2 - self.x1)/2 + self.x1), ((self.y2 - self.y1)/2 + self.y1), text = "Time = "+str(tag)+"s")
        self.canvas.update()
    

if show_animation == True:
    animation = Tk()
    #bitmap = BitmapImage(file="uxbridge.bmp")

    #im = PhotoImage(file="Z:\simulation scheduling\image\uxbridge.png")

    canvas = Canvas(animation, width = 1200, height = 400)
    #canvas.create_image(0,0, anchor=NW, image=im)
    animation.title("Penny fab")

    canvas.pack()

#### matplotlib plots
    

if show_animation == True and hide_plots == False:
    f = plt.Figure(figsize=(5,4), dpi=100)

    a1 = f.add_subplot(221) # cycle time
    a2 = f.add_subplot(222) # throughput
    a3 = f.add_subplot(223) # WIP
    a4 = f.add_subplot(224) # Total wait time

    a1.plot()
    a2.plot()
    a3.plot()
    a4.plot()

    from matplotlib.backends.backend_tkagg import (FigureCanvasTkAgg, NavigationToolbar2Tk)# Implement the default Matplotlib key bindings.
    from matplotlib.backend_bases import key_press_handler
    from matplotlib.figure import Figure
    import numpy as np
    
    dataPlot = FigureCanvasTkAgg(f, master=animation)
    dataPlot.draw()
    dataPlot.get_tk_widget().pack(side=TOP, fill=BOTH, expand=1)
    f.tight_layout()

    canvas.pack()

# platforms
if show_animation == True:
    canvas.create_rectangle(50, 100, 100, 150, fill = "yellow") #job released
    canvas.create_rectangle(250, 100, 300, 150, fill = "yellow") #punch
    canvas.create_rectangle(450, 100, 500, 150, fill = "yellow") #stamp
    canvas.create_rectangle(650, 100, 700, 150, fill = "yellow") #rim
    canvas.create_rectangle(850, 100, 900, 150, fill = "yellow") #deburr
    canvas.create_rectangle(1050, 100, 1100, 150, fill = "yellow") #FP

    canvas.create_rectangle(350, 175, 400, 225, fill='green') #buffer before stamp
    canvas.create_rectangle(550, 175, 600, 225, fill='green') #buffer before stamp
    canvas.create_rectangle(750, 175, 800, 225, fill='green') #buffer before stamp

    canvas.create_line(100, 125, 250, 125, fill="green", width=3) 
    canvas.create_line(300, 125, 450, 125, fill="green", width=3) 
    canvas.create_line(500, 125, 650, 125, fill="green", width=3) 
    canvas.create_line(700, 125, 850, 125, fill="green", width=3)
    canvas.create_line(900, 125, 1050, 125, fill="green", width=3)

    canvas.create_text(50, 75, text = "Job Board")
    canvas.create_text(250, 75, text = "Punch")
    canvas.create_text(450, 75, text = "Stamp")
    canvas.create_text(650, 75, text = "Rim")
    canvas.create_text(850, 75, text = "Deburr")
    canvas.create_text(1050, 75, text = "FP")

    canvas.create_text(350, 250, text = "Buffer 1")
    canvas.create_text(550, 250, text = "Buffer 2")
    canvas.create_text(750, 250, text = "Buffer 3")

############ END OF CANVAS #################
'''pennyfab1 -basic-data to dataframe-with some 
metrics in dataframe'''

import itertools
import random
from random import uniform
import simpy
import pandas as pd


RANDOM_SEED = 42  
hour=8
day=50
SIM_TIME = hour*day            # Simulation time in hour

df=[]

def fab(name, env, punch, stamp, rim, deburr, job_released, FP, df):
    print('Job released for %s at %.1f' % (name, env.now))
    with punch.request() as req:
        start = env.now
        d={'penny_number':name, 'time':env.now}
        yield req
        fab=penny(canvas, 70, 120, 80, 130, name)
        fab.move_penny(200, 0)
        d['started_punch']=env.now
        print ('{} is requested for punch {}'.format(name,env.now))
        yield env.timeout(2)
        d['finished_punch']=env.now
        fab.move_penny(100, 75)
        print ('%s finished punch at %0.1f' % (name,env.now))
        df.append(d)
        
    with stamp.request() as req:
        yield req
        fab.move_penny(100, -75)
        print ('{} is requested for stamp {}'.format(name,env.now))
        d['started_stamp']=env.now
        yield env.timeout(5)
        d['finished_stamp']=env.now
        fab.move_penny(100, 75)
        print ('%s finished stamp at %0.1f' % (name,env.now))
        df.append(d)
        
    with rim.request() as req:
        yield req
        fab.move_penny(100, -75)
        print ('{} is requested for rim {}'.format(name,env.now))
        d['started_rim']=env.now
        yield env.timeout(10)
        d['finished_rim']=env.now
        fab.move_penny(100, 75)
        print ('%s finished rim at %0.1f' % (name,env.now))
        df.append(d)
        
    with deburr.request() as req:
        yield req
        fab.move_penny(100, -75)
        print ('{} is requested for deburr {}'.format(name,env.now))
        d['started_deburr']=env.now
        yield env.timeout(3)
        fab.move_penny(200, 0)
        yield FP.put(1)
        d['finished_deburr']=env.now
        d['wip in punch']=punch.count
        d['Buffer before punch']=len(punch.queue)
        d['wip in stamp']=stamp.count
        d['Buffer before stamp']=len(stamp.queue)
        d['wip in rim']=rim.count
        d['Buffer before rim']=len(rim.queue)
        d['wip in deburr']=deburr.count-1
        d['Buffer before deburr']=len(deburr.queue)
        d['Finished penny']=FP.level
        d['cycle time']=d['finished_deburr']-d['time']
        d['throughput']=d['Finished penny']/d['finished_deburr']
        d['WIP']=punch.count+len(punch.queue)+stamp.count+len(stamp.queue)+rim.count+len(rim.queue)+deburr.count-1+len(deburr.queue)
        d['WT after punch']=d['started_stamp']-d['finished_punch']
        d['WT after stamp']=d['started_rim']-d['finished_stamp']
        d['WT after rim']=d['started_deburr']-d['finished_rim']
        d['total WT']=d['WT after punch']+d['WT after stamp']+d['WT after rim']
        print ('%s finished deburr at %0.1f' % (name,env.now))
        #fab.remove_penny()
        df.append(d)
    
        if show_animation == True and hide_plots == False:
            a1.cla()
            a1.set_xlabel("Number of Penny")
            a1.set_ylabel("Cycle time")
            a1.plot(d['Finished penny'], d['cycle time'], 'ro')

            # setup for headway distribution
            a2.cla()
            a2.set_xlabel("Number of Penny")
            a2.set_ylabel("WIP")
            a2.plot(d['Finished penny'],d['WIP'], 'ro')

            # setup for train count
            a3.cla()
            a3.set_xlabel("Number of Penny")
            a3.set_ylabel("Throughput")
            a3.plot(d['Finished penny'],d['throughput'], 'ro')

            # setup for TPH meter
            a4.cla()
            #TPH = 3600.0/np.mean(headway)
            a4.set_xlabel("Number of Penny")
            a4.set_ylabel("Wait time")
            a4.plot(d['Finished penny'],d['total WT'], 'ro')
        
            dataPlot.draw()
            canvas.update()


def wo_generator(env,punch, stamp, rim, deburr, job_released, FP, df):   
    for i in itertools.count():
        yield env.timeout(1)
        env.process(fab('penny %d' %i, env,punch, stamp, rim, deburr, job_released, FP, df))

def create_clock(env):
    clock = Clock(canvas, 900,350,1100,400, env.now)
    while True:
        yield env.timeout(1)
        clock.tick(env.now)


print('Penny Fab')
random.seed(RANDOM_SEED)
#env=simpy.Environment()
env = simpy.rt.RealtimeEnvironment(factor = 0.1, strict = False)
punch=simpy.Resource(env, capacity= 1) 
stamp=simpy.Resource(env, capacity= 1) 
rim=simpy.Resource(env, capacity=1) 
deburr=simpy.Resource(env, capacity= 1) 

job_released=simpy.Container(env, 500, init=500)
FP=simpy.Container(env, 500, init=0)


env.process(wo_generator(env, punch, stamp, rim, deburr, job_released, FP, df))
env.process(create_clock(env))


env.run(until=SIM_TIME)
mainloop()

data_vis=pd.DataFrame(df)
data_vis=data_vis.drop_duplicates()
#data1['cycle time']=data1['finished_deburr']-data1['started_punch']
#data1['throughput']=data1['Finished penny']/data1['finished_deburr']
#data1['WIP']=data1['Buffer before punch']+data1['Buffer before stamp']+data1['Buffer before rim']+data1['Buffer before deburr']+data1['wip in punch']+data1['wip in stamp']+data1['wip in rim']+data1['wip in deburr']
#data1['WT after punch']=data1['started_stamp']-data1['finished_punch']
#data1['WT after stamp']=data1['started_rim']-data1['finished_stamp']
#data1['WT after rim']=data1['started_deburr']-data1['finished_rim']
#data1['total WT']=data1['WT after punch']+data1['WT after stamp']+data1['WT after rim']



data_vis.to_csv("data_vis.csv", sep=',',index=False)



at 198.0
penny 18 is requested for deburr 198
penny 19 is requested for rim 198
penny 39 is requested for stamp 198
penny 98 finished punch at 199.0
Job released for penny 198 at 199.0
penny 99 is requested for punch 199
Job released for penny 199 at 200.0
penny 99 finished punch at 201.0
Job released for penny 200 at 201.0
penny 18 finished deburr at 201.0
penny 100 is requested for punch 201
Job released for penny 201 at 202.0
penny 39 finished stamp at 203.0
penny 100 finished punch at 203.0
Job released for penny 202 at 203.0
penny 40 is requested for stamp 203
penny 101 is requested for punch 203
Job released for penny 203 at 204.0
penny 101 finished punch at 205.0
Job released for penny 204 at 205.0
penny 102 is requested for punch 205
Job released for penny 205 at 206.0
penny 102 finished punch at 207.0
Job released for penny 206 at 207.0
penny 103 is requested for punch 207
penny 19 finished rim at 208.0
penny 40 finished stamp at 208.0
Job released for penny 207 at 208.0
penny